In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 🧠 Problem Type

This is a **Supervised Learning** problem — specifically **Regression** (no classification drama here).

---

## 🎯 Target Variable

Our precious target column is:

> **`exam_score`**
> (the final boss we’re trying to predict)

---

## 📏 Evaluation Metric

We’ll judge our model using:

> **RMSE (Root Mean Squared Error)**

Why RMSE?

* Penalizes big mistakes harder (strict teacher vibes 👀)
* Perfect for continuous values like exam scores

---

## 📐 Dataset Shape

The dataset is *not small, not medium… it’s Kaggle-big* 💪

```
Rows × Columns = 630,000 × 13
```

That’s:

* Enough data to train a serious model
* Enough data to accidentally mess up RAM if not careful 😅

---

## 🚨 Data Leakage (The Crime Scene 🕵️‍♀️)

**Data leakage is when your model secretly cheats in the exam**
— just like knowing the answer key before the test 📝❌

Examples of leakage we *absolutely avoided*:

* ❌ Using future information
* ❌ Letting `exam_score` influence features
* ❌ Splitting data *after* feature engineering

### 🔐 The Suspect: `Id`

The `Id` column tried to act innocent, but…

> **Id knows nothing about exam scores.
> It’s just a roll number pretending to be useful.**

So we politely showed it the exit 🚪🙂

---

Now let the model **study properly** and earn its marks 📚🔥

---


In [ ]:

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

In [ ]:
# Load dataset 
df = pd.read_csv('/kaggle/input/playground-series-s6e1/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s6e1/test.csv')

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe()

In [ ]:
# plot histogram of numeric cols
df.hist(bins=50,figsize=(25,20))
plt.show()

In [ ]:
df['high_att+study']= ((df['class_attendance']>=90) &(df['study_hours']>=6)).astype(int)
df['ideal_sleep'] = ((df['sleep_hours']>=7) & (df['sleep_hours']<=9)).astype(int)
df['ideal_study'] = (df['study_hours']>=7).astype(int)

In [ ]:
df.columns

In [ ]:
X = df.drop(columns=['id','exam_score'])
y = df['exam_score']

In [ ]:
# split data
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.2,random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
# seperate numerical and categorical columns
num_col = pd.DataFrame(X_train.select_dtypes(include=['int64','float64']))
cat_col = X_train.select_dtypes(include=['object']).columns

In [ ]:
# value_count for categorical data
for col in cat_col:
    print(df[col].value_counts())

In [ ]:
num_col.corrwith(y_train)

In [ ]:
# import seaborn as sns 
# sns.pairplot(num_col)
# plt.show()

In [ ]:
# for col in num_col:
#     plt.figure(figsize=(30,10))
#     sns.boxplot(x=num_col[col],y=y_train)
#     plt.show

In [ ]:
ordinal_col = ['internet_access','sleep_quality','facility_rating','exam_difficulty']

normina_col = [col for col in cat_col if col not in ordinal_col ]

## 🚨 Outlier Handling — *Standard Deviation Edition*

Some features decided to go **too far from the mean** and risk ruining our RMSE.
Instead of panicking, we handled them **statistically and calmly** 🧘‍♀️

---

### 🧪 Strategy Used: **Standard Deviation–Based Capping**

Outliers are capped using the formula:

[
{Lower Bound} = \mu - k\sigma \
{Upper Bound} = \mu + k\sigma
]

Where:

* (\mu) = feature mean
* (\sigma) = feature standard deviation
* (k = 3) (default)

This assumes features are **approximately normally distributed** — fair enough for exam-related data 📊

---

### 🛠️ Implementation Details

* A **custom scikit-learn transformer** (`CappingTransformer`) was created
* Compatible with **pipelines & cross-validation** ✔️
* Bounds are **learned only from training data** (no cheating 👀)
* Same bounds are applied consistently during inference

---

### ✂️ What Exactly Happens?

* ❌ No rows are dropped
* ❌ No data leakage
* ✅ Values beyond ±3σ are **clipped**
* 🎯 Target column (`exam_score`) is **not transformed**

Outliers were told:

> “You’re still part of the dataset — just stay within 3σ.”

---

### 🤔 Why Capping Instead of Removal?

* Large dataset (**630k rows**) → keep information
* Prevents extreme values from dominating loss
* RMSE stays stable
* Model stays sane

---

Data is now **clean, disciplined, and model-ready** 🚀

---



In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CappingTransformer(BaseEstimator, TransformerMixin):
    """
    A custom transformer for capping outliers in a normal distribution 
    based on standard deviations within a scikit-learn pipeline.
    
    Attributes:
        std_devs (float): The number of standard deviations to use for bounds.
        upper_bounds_ (dict): Stored upper bounds for each feature during fit.
        lower_bounds_ (dict): Stored lower bounds for each feature during fit.
    """
    def __init__(self, std_devs=3.0):
        self.std_devs = std_devs
        self.upper_bounds_ = {}
        self.lower_bounds_ = {}

    def fit(self, X, y=None):
        """Calculates the capping bounds based on the input data."""
        # Check if X is a pandas DataFrame and convert to numpy array if not for consistency
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            
        for col in X.columns:
            mean = X[col].mean()
            std = X[col].std()
            self.upper_bounds_[col] = mean + (self.std_devs * std)
            self.lower_bounds_[col] = mean - (self.std_devs * std)
            
        return self

    def transform(self, X):
        """Applies the capping bounds to the input data."""
        # Ensure transformation data matches the structure of fitting data
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            
        X_capped = X.copy()
        for col in X_capped.columns:
            # Clip values: values above upper bound become the upper bound, 
            # and values below the lower bound become the lower bound.
            X_capped[col] = np.clip(X_capped[col], 
                                    self.lower_bounds_[col], 
                                    self.upper_bounds_[col])
            
        return X_capped


In [ ]:
outlier = ['study_hours','class_attendance','ideal_study','high_att+study']

In [ ]:
num_pipe = Pipeline([
    ('imputer',SimpleImputer(strategy='mean')),
    ('scale',StandardScaler())
])

ord_pipe =Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OrdinalEncoder(
        categories=[
         
                    ['yes','no'],
                    ['good','average','poor'],
                    ['high','medium','low'],
                    ['easy','moderate','hard']
         
        ]
    ))
])

nor_pipe = Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])

out_pipe = Pipeline([
     ('capper', CappingTransformer(std_devs=3.0)),
])

In [ ]:
num_col = X_train.select_dtypes(include=['int64','float64']).columns

In [ ]:
preprocessor = ColumnTransformer([
    ('out', out_pipe, outlier),
    ('num', num_pipe, num_col),
    ('ord', ord_pipe, ordinal_col),
    ('nom', nor_pipe, normina_col)
])

## 🤖 Model — LightGBM Regressor (The Workhorse)

For this task, we use **LightGBM**, a gradient boosting framework known for:

* High performance on large tabular datasets
* Fast training
* Strong handling of non-linear relationships

Perfect fit for **630k rows** 💪

---

### 🧩 Pipeline Setup

The model is wrapped inside a **scikit-learn Pipeline**, ensuring:

* All preprocessing happens **inside CV / train-val split**
* Zero data leakage
* Clean, reproducible workflow

Pipeline structure:

* **Preprocessor** → feature transformations
* **LGBMRegressor** → prediction engine

---

### ⚙️ Model Configuration (Key Choices)

* **n_estimators = 5000**
  Allows the model to learn complex patterns with a small learning rate

* **learning_rate = 0.03**
  Slow and steady learning = better generalization

* **max_depth = 5**
  Controls overfitting by limiting tree complexity

* **num_leaves = 31**
  Balanced expressive power without becoming noisy

* **min_data_in_leaf = 40**
  Prevents the model from memorizing tiny patterns

---

### 🛡️ Regularization & Stability

To keep the model disciplined:

* **L1 Regularization (`lambda_l1 = 0.2`)**
* **L2 Regularization (`lambda_l2 = 0.4`)**
* **Feature sampling (`feature_fraction = 0.85`)**
* **Row sampling (`bagging_fraction = 0.85`, `bagging_freq = 1`)**

Translation:

> The model is smart, not reckless 😌

---

### 📏 Evaluation Metric

Model performance is evaluated using:

> **RMSE (Root Mean Squared Error)**

* Computed on both **training** and **validation** sets
* Helps monitor overfitting
* Penalizes large prediction errors

---

### 📉 Results Interpretation

* **Train RMSE** → how well the model fits known data
* **Validation RMSE** → how well it generalizes

A small gap between the two indicates:

* Good generalization
* No major overfitting

---

Model is **trained, tested, and ready to compete** 🚀

---


In [ ]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

rf_model = Pipeline([
    ('prep',preprocessor),
    ('model',LGBMRegressor(
        n_estimators=5000,
        learning_rate=0.03,
        max_depth=5,
        num_leaves=31,
        min_data_in_leaf=40,
        lambda_l1=0.2,
        lambda_l2=0.4,
        feature_fraction=0.85,
        bagging_fraction=0.85,
        bagging_freq=1,
        random_state=42,
        n_jobs=-1,
        verbosity=-1))

])

rf_model.fit(X_train, y_train)


y_test_prob = rf_model.predict(X_val)
y_train_prob = rf_model.predict(X_train)

rmse_val =  np.sqrt(mean_squared_error(y_val, y_test_prob))
rmse_train =  np.sqrt(mean_squared_error(y_train, y_train_prob))

print("Train RMSE:", rmse_train)
print("Test RMSE:", rmse_val)

In [ ]:
test['high_att+study'] = (
    (test['class_attendance'] >= 90) &
    (test['study_hours'] >= 6)
).astype(int)

test['ideal_sleep'] = (
    (test['sleep_hours'] >= 7) &
    (test['sleep_hours'] <= 9)
).astype(int)

test['ideal_study'] = (
    test['study_hours'] >= 7
).astype(int)

In [ ]:
test_proba = rf_model.predict(test)
submission = pd.DataFrame({
    'id': test['id'],
    'exam_score':test_proba
})
submission.to_csv('submission3.csv', index=False)

## 🏁 Final Results & Looking for Feedback

After extensive experimentation and tuning, the **best score I’ve been able to achieve** in this competition is:

> **🏆 RMSE: 8.68897**

---

### 🔁 What I’ve Tried So Far

To push the score further, I experimented with:

* LightGBM hyperparameter tuning
* Different preprocessing strategies
* Feature engineering ideas
* Regularization adjustments
* Ensembling techniques

Despite these efforts, I haven’t been able to achieve any **meaningful improvement** beyond this point.

---

### 📉 Current Situation

At the moment, it feels like I’ve hit a **performance plateau**.
Further tuning either gives negligible gains or makes validation unstable.

This makes me think:

* I may be missing some **important feature insight**
* Or there might be **data limitations** I’m not seeing clearly

---

### 🧠 About My Approach

I’ve focused purely on **traditional machine learning models**, mainly tree-based methods like LightGBM.
I haven’t explored deep learning yet, as my current knowledge is limited to ML, and DL feels out of reach for now.

---

### 🙏 Asking for Help

I’d genuinely appreciate feedback from the community on:

* Feature engineering ideas I might have missed
* Validation strategy improvements
* Whether this plateau is expected for this dataset
* Any LightGBM tuning suggestions worth trying

If you see something obvious (or non-obvious!) that I overlooked, I’d love to learn from it.

---

### ✅ Closing Note

This notebook represents my **best effort so far**, but I know there’s room to grow.
Any constructive feedback or suggestions would mean a lot.

Thank you for taking the time to read 🙌
